In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
def ks_table_summary(prediction,actual,bin):
    #prediction is [:,1] of predicted probability column
    #prob = pd.DataFrame(prediction)
#   prob = prob.reset_index()
    prob = prediction
    target = actual
#   target = pd.DataFrame(actual)
#   target = target.reset_index()
#   target = target.iloc[:,~0]
    
    qv = prob.quantile(np.linspace(0.0,1.0,num=bin),interpolation = 'linear')
    prob_bins = pd.cut(prob,list(sorted(set(qv))),include_lowest = True)
    df = pd.concat([prob_bins,prob,target],axis = 1)
    df.columns = ['prob_bins','prob','target']
    
    grp = df.groupby('prob_bins')
    account = grp['target'].count().astype('float')
    event = grp['target'].sum().astype('float')
    non_event = account-event 
    actual_non_event_rate = 1-grp['target'].mean().astype('float')
    actual_event_rate = grp['target'].mean().astype('float')
    grp = df.groupby('prob_bins')
    min_score = grp['prob'].min().astype('float')
    max_score = grp['prob'].max().astype('float')
    pred_event_rate = grp['prob'].mean().astype('float')
    
    ks_tab = pd.concat([account,non_event,event,actual_non_event_rate,actual_event_rate,min_score,max_score,pred_event_rate],axis = 1)
    ks_tab.columns = ['account','non_event','event','actual_non_event_rate','actual_event_rate','min_score','max_score','pred_event_rate']
    
    ks_tab['pred_event'] = ks_tab['account']*ks_tab['pred_event_rate']
    ks_tab['pred_event'] = ks_tab['pred_event'].astype('int')
    ks_tab['non_event_capture_rate'] = ks_tab['non_event']/ks_tab['non_event'].sum()
    ks_tab['event_capture_rate'] = ks_tab['event']/ks_tab['event'].sum()
    
    cum_list1 = []
    cum_list2 = []
    lst = list(ks_tab['event_capture_rate'])
    length = len(lst)
    cum_list1 = [sum(lst[(length - x - 1):length]) for x in range(0,length)]
    lst = list(ks_tab['non_event_capture_rate'])
    cum_list2 = [sum(lst[(length - x - 1):length]) for x in range(0,length)]
    cum_list1.reverse()
    cum_list2.reverse()
    ks_tab['event_cum_capture_rate'] = cum_list1
    ks_tab['non_event_cum_capture_rate'] = cum_list2
    ks_tab['KS_Stat'] = ks_tab['event_cum_capture_rate']-ks_tab['non_event_cum_capture_rate']
    ks_tab['KS_Stat'] = np.round(ks_tab['KS_Stat'].astype(float),3)
    
    return ks_tab

In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/deepanshu88/data/master/data.csv")


In [ ]:
ks_table_summary(df['p'],df['y'],11)

**One Sampe Kolmogorov-Smirnov Test**

The one-sample Kolmogorov-Smirnov test is used to test whether a sample comes from a specific distribution. We can use this procedure to determine whether a sample comes from a population that is normally distributed 

Null hypothesis assumes that the numbers are uniformly distributed between 0-1.

In [ ]:
from scipy.stats import kstest
import random
actual = np.random.randint(0,100,size = 100)
x = kstest(actual, "uniform")   
print(x)

In [ ]:
#Null hypothesis assumes that the numbers are Normally distributed
from scipy.stats import kstest
import random
actual = np.random.randint(0,100,size = 100)
x = kstest(actual, "norm")   
print(x)

**Two-Sample Kolmogorov-Smirnov Test**

The null hypothesis is H0: both samples come from a population with the same distribution. 

In [ ]:
from scipy.stats import ks_2samp
df = pd.read_csv("https://raw.githubusercontent.com/deepanshu88/data/master/data.csv")
ks_2samp(df.loc[df.y==0,"p"], df.loc[df.y==1,"p"])

It returns KS score 0.6033 and p-value less than 0.01 which means we can reject the null hypothesis and concluding distribution of events and non-events is different.